In [1]:
# The idea is to create a WaveNet
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import nn
%matplotlib inline

In [2]:
# Prepping Data Set and Data Loader:
#Data:
names = open("names.txt").read().splitlines()
print(f"{len(names)=}")

# Build Vocab
itos = {i+1:s for i,s in enumerate(sorted(set("".join(names))))}
itos[0] = '.'

num_embeddings = len(itos)
print(num_embeddings)
stoi = {s:i for i,s in itos.items()}

len(names)=32033
27


In [3]:
#Data Set Creation:
def create_data(names):
    x, y = [], []
    context_len = 8
    xchar, ychar = [], []
    buff = [0]*context_len
    buff = [itos[b] for b in buff]
    for name in names:
        augmented_name = buff + list(name) + ['.']
        for i in range(len(augmented_name)-context_len):
            xi = augmented_name[i:i+context_len ]
            yi = augmented_name[i+context_len]
            #print(f"{xi} -------> {yi}")
            xchar.append(xi)
            ychar.append(yi)
            x.append([stoi[d] for d in xi])
            y.append(stoi[yi])
    
    return torch.tensor(x), torch.tensor(y)


In [4]:
#Create Minibathces:
import random
random.shuffle(names)

train = int(0.8 * len(names))
val = int(0.1 * len(names))
test = int(0.1 * len(names))

train, test, val

(25626, 3203, 3203)

In [5]:
Xtr, Ytr = create_data(names[:train])
Xval, Yval = create_data(names[train:train+val])
Xtest, Ytest = create_data(names[train+val:train+val+test])

In [6]:
for x,y in zip(Xtr[:20], Ytr[:20]):
  print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

........ --> a
.......a --> l
......al --> o
.....alo --> n
....alon --> a
...alona --> h
..alonah --> .
........ --> k
.......k --> e
......ke --> i
.....kei --> a
....keia --> r
...keiar --> a
..keiara --> .
........ --> o
.......o --> l
......ol --> u
.....olu --> w
....oluw --> a
...oluwa --> t


In [7]:
# hierarchical network
n_embd = 24 # the dimensionality of the character embedding vectors
n_hidden = 128 # the number of neurons in the hidden layer of the MLP
model = nn.Sequential([
  nn.Embedding(num_embeddings, n_embd),
  nn.FlattenConsecutive(2), nn.Linear(n_embd * 2, n_hidden, bias=False), nn.BatchNorm1d(n_hidden), nn.Tanh(),
  nn.FlattenConsecutive(2), nn.Linear(n_hidden*2, n_hidden, bias=False), nn.BatchNorm1d(n_hidden), nn.Tanh(),
  nn.FlattenConsecutive(2), nn.Linear(n_hidden*2, n_hidden, bias=False), nn.BatchNorm1d(n_hidden), nn.Tanh(),
  nn.Linear(n_hidden, num_embeddings),
])

# parameter init
with torch.no_grad():
  model.layers[-1].weight *= 0.1 # last layer make less confident

parameters = model.parameters()
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

76579
